## DCF_1 ###

- Basic version of DCF 
- Acquire data from Yahoo finance using "yfinance"


step :
- get > Average Free cash flow , shares , Net Debt = Long Term Borrowings - Cash
- input > growth rate , terminal growth rate , discount rate , horizons
- calculate > FCF 
- calculate > terminal value
- calculate > present value
- calculate > Intrinsic value
- calculate > Margin of safety
- summary

In [2]:
import pandas as pd
import yfinance as yf

In [3]:
ticker = 'mega.bk'
data = yf.Ticker(ticker)
data.info

429 Client Error: Too Many Requests for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/MEGA.BK?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=MEGA.BK&crumb=Edge%3A+Too+Many+Requests


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
### - get > Average Free cash flow , shares , Net Debt = Long Term Borrowings - Cash ###

## Shares ## 
shares = data.info['sharesOutstanding']
print('Ticker = ',ticker,' Shares = ',shares)


## Net Debt = Long Term Borrowings - Cash ###
totalnoncurrentliabilities = data.balancesheet.loc[data.balancesheet.index == 'Total Non Current Liabilities Net Minority Interest'].values[0][0] - data.balancesheet.loc[data.balancesheet.index == 'Cash And Cash Equivalents'].values[0][0]
print('Ticker = ',ticker,' Net Debt = ',totalnoncurrentliabilities)

## Average Free Cash Flow ## 
averagefreecashflow = data.cash_flow.T['Free Cash Flow'].dropna().mean()
print(data.cash_flow.T['Free Cash Flow'].dropna())
print('Ticker = ',ticker,'Average Free Cash Flow = ',averagefreecashflow)

Ticker =  mega.bk  Shares =  871870016
Ticker =  mega.bk  Net Debt =  -1980612000.0
2023-12-31    1632912000.0
2022-12-31    1683881000.0
2021-12-31    2567048000.0
2020-12-31    1374361000.0
Name: Free Cash Flow, dtype: object
Ticker =  mega.bk Average Free Cash Flow =  1814550500.0


In [ ]:
## - input > growth rate , terminal growth rate , discount rate , horizons ##
FCF = data.cash_flow.T['Free Cash Flow'].dropna()
FCF = pd.DataFrame(FCF)
FCF = FCF.sort_index(ascending=True)
FCF['diff'] = FCF['Free Cash Flow'].diff(1)
FCF['growth'] = FCF['diff']/FCF['Free Cash Flow']
print(FCF)
growthrate = FCF['growth'].mean()
print("Average Growth Rate from Operating Cash Flow = ",growthrate)

year_growth = 3
year_sustain = 7
discountrate = 0.10
terminalgrowthrate = 0.02
growthrate = 0.5

           Free Cash Flow          diff    growth
2020-12-31   1374361000.0           NaN       NaN
2021-12-31   2567048000.0  1192687000.0  0.464614
2022-12-31   1683881000.0  -883167000.0 -0.524483
2023-12-31   1632912000.0   -50969000.0 -0.031214
Average Growth Rate from Operating Cash Flow =  -0.030360789435253045


In [ ]:
 ## - calculate > FCF ##

FCF_year_growth = [averagefreecashflow * ((1+growthrate)**i) for i in range(1,year_growth+1,1)]
print("FCF year growth",FCF_year_growth)

FCF_year_sustain = [FCF_year_growth[-1] * ((1+(growthrate/2))**i) for i in range(1,year_sustain+1,1)]
print("FCF year sustain",FCF_year_sustain)

## calculate > terminal value
terminalvalue = FCF_year_sustain[-1] * (1+(terminalgrowthrate)) / (discountrate-terminalgrowthrate)
print(FCF_year_sustain[-1]* (1+(terminalgrowthrate)))
print("Terminal value = ",terminalvalue)


FCF year growth [np.float64(2721825750.0), np.float64(4082738625.0), np.float64(6124107937.5)]
FCF year sustain [np.float64(7655134921.875), np.float64(9568918652.34375), np.float64(11961148315.429688), np.float64(14951435394.28711), np.float64(18689294242.858887), np.float64(23361617803.57361), np.float64(29202022254.46701)]
29786062699.55635
Terminal value =  372325783744.45435


In [ ]:
## - calculate > present value ##
FCF_year_growth = FCF_year_growth + (FCF_year_sustain)
FCF_year_growth.append(terminalvalue)
pv = pd.DataFrame(FCF_year_growth,columns=['FCF'])
pv['pv'] = [pv['FCF'][i]/((1+discountrate)**(i+1)) for i in range(len(pv))]
pv['pv'][pv.index[-1]] = pv['FCF'][pv.index[-1]] / ((1+discountrate)**(len(pv)-1))
print(pv)

totalpv = pv['pv'].sum() - totalnoncurrentliabilities
print('total pv = ',totalpv)

intrinsicvalue = totalpv/shares
print('intrinsicvalue = ',intrinsicvalue)

             FCF            pv
0   2.721826e+09  2.474387e+09
1   4.082739e+09  3.374164e+09
2   6.124108e+09  4.601133e+09
3   7.655135e+09  5.228560e+09
4   9.568919e+09  5.941546e+09
5   1.196115e+10  6.751756e+09
6   1.495144e+10  7.672450e+09
7   1.868929e+10  8.718694e+09
8   2.336162e+10  9.907606e+09
9   2.920202e+10  1.125864e+10
10  3.723258e+11  1.435477e+11
total pv =  211457260056.86676
intrinsicvalue =  242.53301085751153


C:\Users\595078\AppData\Local\Temp\ipykernel_18244\3552423011.py:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  pv['pv'][pv.index[-1]] = pv['FCF'][pv.index[-1]] / ((1+discountrate)**(len(pv)-1))
